In [ ]:
import os

import pandas

import arcpy
import arcgis
from arcgis.features import GeoAccessor

import jinja2
try:
    import pdfkit
    create_pdfs = True
except:
    create_pdfs = False
    print("Unable to import pdfkit, pdfs will not be generated.")
    print("Code will only produce .html files instead.")

In [ ]:
SSP_offices_path = r"C:\SSP_offices"

In [ ]:
def create_PDFs(input_office_name):
    """Creates a pdf for the specified office."""
    print(f"Creating PDF for Office: {input_office_name}")
    ## the aprx file we created in ArcGIS Pro
    aprx = arcpy.mp.ArcGISProject(os.path.join(SSP_offices_path,"APRX\SSP_offices\SSP_offices.aprx"))
    ## the directory where our HTML template is located
    template_directory = os.path.join(SSP_offices_path,"HTML_template")
    ## the name of the HTML template file
    HTML_template_name = "OfficeInfo.html"
    ## the directory where the PDFs will be created
    ouput_dir = r"C:\SSP_offices\pdfs"
    ## The geodatabase that has the feature class that the report will be generated from
    gdb = os.path.join(SSP_offices_path,"APRX\SSP_offices\Default.gdb")
    ## the name of the Feature class that the report will be generated from
    fc_name = "OFFICE"
    ## the field name that contains the unique name for the 
    field_name = "OfficeName"

    ## our APRX only has one map, so get the first map in the list
    mapx = aprx.listMaps()[0]
    ## same for layouts, only one, get the frist one
    lyt = aprx.listLayouts()[0]
    ## get the mapframe from the layout, once again, only one map frame... so get the first one
    mf = lyt.listElements('MAPFRAME_ELEMENT','*')[0]

    ## load the directory that holds the directory
    templateLoader = jinja2.FileSystemLoader(searchpath=template_directory)
    ## set the environment
    templateEnv = jinja2.Environment(loader=templateLoader)
    ## get the template HTML file
    template = templateEnv.get_template(HTML_template_name)

    ## iterate over a data frame (that only has one row because of the whereclause) 
    ## to get the row with the specified name, this could be cahnged to work with a search cursor too
    ## but I really like working with dataframes
    wc = f"{field_name} = '{input_office_name}'"
    for row in GeoAccessor.from_featureclass(os.path.join(gdb, fc_name), where_clause=wc).itertuples():
        ## find the lat/long of the point and move the mapframe to that coordinate and set the scale
        lat = row.SHAPE.y
        long = row.SHAPE.x
        mf.camera.X = long
        mf.camera.Y = lat
        mf.camera.scale = 5000
        ## export the layout to a .png
        img_path = os.path.join(ouput_dir, f"{input_office_name}_map.png")
        lyt.exportToPNG(img_path)
        
        ## create a min/max temperature plot
        ## this demo is not meant to be a lesson in plotting with pandas, 
        ## this is simply showing that we can get related records from a different table
        ## plot them, save it and bring it into the report
        df = GeoAccessor.from_table(os.path.join(gdb, "WEATHER"), where_clause=wc)
        df.set_index("Date", inplace=True)
        df.drop(columns="OBJECTID", inplace=True)
        line_colors = {"MaxTemperature":"Red", "MinTemperature":"Blue"}
        temp_plot = df.loc[df["OfficeName"]==input_office_name].plot(ylim=(0,120), color=line_colors, ylabel="Temperature °F", figsize=(10, 6))
        temp_plt_path = os.path.join(ouput_dir, f"{input_office_name}_temp_plot.png")
        temp_plot.get_figure().savefig(temp_plt_path)

        ## use the jinja2 template to render the template with our values
        outputText = template.render(
                                     OfficeName=row.OfficeName,
                                     Phone=row.Phone,
                                     MapViewTag=f"<img src='{img_path}'>", ## note we're injecting the whole HTML tag for the image here
                                     StreetAddress=row.StreetAddress,
                                     City=row.City,
                                     State=row.State,
                                     PostalCode=row.PostalCode,
                                     County=row.County,
                                     LAT=f"{lat:.5f}", ## round the lat and long to 5 decimal places for consistency 
                                     LONG=f"{long:.5f}",
                                     TemperaturePlotTag=f"<img src='{temp_plt_path}'>", ## injecting the whole HTML tag for the image here too
                                    )
        ## open a file and write the HTML text to the file
        html_file_path = os.path.join(ouput_dir,f'{input_office_name}.html')
        html_file = open(html_file_path, 'w')
        html_file.write(outputText)
        html_file.close()
        if not create_pdfs:
            ## since we couldnt import pdfkit the code will not attempt to conver the .html to a .pdf
            print(f"Created HTML successfully: {html_file_path}")
        
        if create_pdfs:
            ## the enable-local-file-access option will allow pdfkit to access the .pngs we exported from the APRX and graph
            ## the quiet option will keep pdfkit from printing a bunch of status messages while it is making the PDF
            options = {
              "enable-local-file-access": None,
              "quiet": ""
            }
            ## turn the HTML file into a PDF
            pdf_file_path = os.path.join(ouput_dir,f'{input_office_name}.pdf')
            pdfkit.from_file(html_file_path, pdf_file_path, options=options)
            ## delete the .pngs and .html
            os.remove(img_path)
            os.remove(temp_plt_path)
            os.remove(os.path.join(ouput_dir,f'{input_office_name}.html'))
            print(f"Created PDF successfully: {pdf_file_path}")
    return True

In [ ]:
create_PDFs("Corporate Headquarters")
create_PDFs("Wisonsin")